# Imports

In [1]:
from PIL import Image, ImageOps, ImageDraw
from bresenham import bresenham
from skimage.color import rgb2gray
from skimage.draw import line
import numpy as np
import matplotlib.pyplot as plt
import sys

# Parameters

In [127]:
# Parameters 
INPUT_IMAGE_PATH = "taco.png"
OUTPUT_IMAGE_PATH = "taco_threaded.png"
OUTPUT_IMAGE_DIMENSIONS = (1000, 1000)
OUTPUT_IMAGE_RADIUS = OUTPUT_IMAGE_DIMENSIONS[0] / 2 - 0.5 # -0.5 to make it exactly on the center
OUTPUT_IMAGE_CENTER_POSITION = (OUTPUT_IMAGE_DIMENSIONS[0] / 2, OUTPUT_IMAGE_DIMENSIONS[1] / 2)
NAILS_AMOUNT = 100
SKIP_NEIGHBOUR_NAILS = 1

print("Parameters:")
print("INPUT_IMAGE_PATH: " + INPUT_IMAGE_PATH)
print("OUTPUT_IMAGE_PATH: " + OUTPUT_IMAGE_PATH)
print("OUTPUT_IMAGE_DIMENSIONS: " + str(OUTPUT_IMAGE_DIMENSIONS))
print("OUTPUT_IMAGE_RADIUS: " + str(OUTPUT_IMAGE_RADIUS))
print("OUTPUT_IMAGE_CENTER_POSITION: " + str(OUTPUT_IMAGE_CENTER_POSITION))
print("NAILS_AMOUNT: " + str(NAILS_AMOUNT))
print("SKIP_NEIGHBOUR_NAILS: " + str(SKIP_NEIGHBOUR_NAILS))

Parameters:
INPUT_IMAGE_PATH: taco.png
OUTPUT_IMAGE_PATH: taco_threaded.png
OUTPUT_IMAGE_DIMENSIONS: (1000, 1000)
OUTPUT_IMAGE_RADIUS: 499.5
OUTPUT_IMAGE_CENTER_POSITION: (500.0, 500.0)
NAILS_AMOUNT: 100
SKIP_NEIGHBOUR_NAILS: 1


In [133]:
def get_input_image():
    # Load image as grayscale
    image = Image.open(INPUT_IMAGE_PATH).convert('LA')
    
    # Crop image to a square
    width, height = image.size
    if width > height:
        left = (width - height) / 2
        right = left + height
        top = 0
        bottom = height
    else:
        left = 0
        right = width
        top = (height - width) / 2
        bottom = top + width
    image.crop((left, top, right, bottom))

    # Resize image
    image = image.resize(OUTPUT_IMAGE_DIMENSIONS)

    # Crop image to a circle
    mask = Image.new('L', image.size, 0)
    draw = ImageDraw.Draw(mask) 
    draw.ellipse((0, 0) + image.size, fill=255)
    output = ImageOps.fit(image, mask.size, centering=(0.5, 0.5))
    output.putalpha(mask)

    # display(output)

#     # Convert image to grayscale withouth alpha
#     no_alpha_image = []
#     display(np.array(image))
#     for pixel in image.getdata():
#         if pixel[1] == 0:
#             no_alpha_image.append((0, 0))
#         else:
#             no_alpha_image.append(pixel)


    new_image = []
    # display(np.array(image))
    for pixel in output.getdata():
        if pixel[1] == 0:
            new_image.append((0, 0))
        else:
            new_image.append(pixel)
    g = image.copy()
    g.putdata(new_image)


    # return g
    
#     g = output.copy()
#     e = g.putdata(no_alpha_image)

    # display(np.array(g.convert('L')))
    # display(g.convert('L'))
    
    return image

img = get_input_image()
print(img.size)
# display(img)
# print(np.array(img))


(1000, 1000)


In [129]:


# Display a list of lines on a given image
def display_threads(lines, image):
    canvas = image.copy()
    for line in lines:
        draw = ImageDraw.Draw(canvas)
        draw.line((line[0][0], line[0][1], line[1][0], line[1][1]), fill=255, width=1)
    display(canvas)


# Display a list of points on a given image
def display_points(points, image):
    canvas = image.copy()
    for point in points:
        canvas.putpixel((int(point[0]), int(point[1])), 255)
    display(canvas)
    
    
    
    
    
    
def generate_new_image(dimensions):
    return Image.new('L', dimensions, 0)
    
def generate_nail_positions(radius, center, amount):
    nails = []
    for i in range(0, amount):
        angle = (2 * np.pi * i / amount) - np.pi / 2 # -pi/2 to make it start at the top
        x = radius * np.cos(angle) + center[0]
        y = radius * np.sin(angle) + center[1]
        nails.append((int(x), int(y)))
    return nails

def generate_thread_positions(nails):
    threads = []
    for i in range(0, len(nails)):
        for j in range(i + 1 + SKIP_NEIGHBOUR_NAILS, len(nails)):
            if i == 0 and j >= len(nails) - SKIP_NEIGHBOUR_NAILS: continue # Skip the last nails in first iteration
            threads.append((nails[i], nails[j]))
    return threads

def get_threads_from_nail(threads, nail):
    threads_from_nail = []
    for thread in threads:
        if nail in thread:
            threads_from_nail.append(thread)
    return threads_from_nail

def generate_nail_to_threads_dictionary(nails, threads):
    nail_to_thread_dictionary = {}
    for nail in nails:
        nail_to_thread_dictionary[nail] = get_threads_from_nail(threads, nail)
    return nail_to_thread_dictionary

def generate_thread_to_points_dictionary(threads):
    thread_to_points_dictionary = {}
    for thread in threads:
        a, b = line(thread[0][0], thread[0][1], thread[1][0], thread[1][1])
        points = []
        for i in range(len(a)):
            points.append((a[i], b[i]))
        thread_to_points_dictionary[thread] = points

    return thread_to_points_dictionary


# img = generate_new_image((28, 28))
# n = generate_nail_positions(13.5, (14, 14), 6)
# t = generate_thread_positions(n)
# ntt = generate_nail_to_threads_dictionary(n, t)
# ttp = generate_thread_to_points_dictionary(t)

# display(img)
# display_points(n, img)
# display_threads(t, img)

# test = img.copy()
# for x in ttp[t[3]]:
#     print(x)
#     test.putpixel(x, (255))
# display(test)



def get_other_end_of_thread(thread, nail):
    if nail == thread[0]:
        return thread[1]
    else:
        return thread[0]

# dictionary[thread] = ((all, coords), (of, the), (threads, that), (start, at), (nail, nail))


# Get all threads from a given nail



# def remove_hidden_values(image):
#     new_image = []
#     display(np.array(image))
#     for pixel in image.getdata():
#         if pixel[1] == 0:
#             new_image.append((0, 0))
#         else:
#             new_image.append(pixel)
#     g = image.copy()
#     g.putdata(new_image)
#     return g




# Draw white line on the image
def draw_line(image, line):
    draw = ImageDraw.Draw(image)
    draw.line((line[0][0], line[0][1], line[1][0], line[1][1]), fill=255, width=1)
    return image


def get_sum_of_values_of_no_alpha_pixels(image):
    sum = 0
    for pixel in image.getdata():
        if pixel[1] == 255:
            sum += pixel[0]
    return sum

def get_amount_of_no_alpha_pixels(image):
    sum = 0
    for pixel in image.getdata():
        if pixel[1] == 255:
            sum += 1
    return sum

def get_image_fitness(image):
    a = np.sum(image) / (np.pi * OUTPUT_IMAGE_RADIUS ** 2)
    return get_sum_of_values_of_no_alpha_pixels(image) / get_amount_of_no_alpha_pixels(image)

    test = Image.new('LA', (3, 3), (255, 0))
    test.putpixel((0, 0), (255, 255))
    test.putpixel((1, 0), (0, 0))
    test.putpixel((2, 0), (0, 255))
    test.putpixel((0, 1), (255, 0))
#     display(test)
#     display(np.array(test))
    
    sum = get_sum_of_values_of_no_alpha_pixels(test)
    amount = get_amount_of_no_alpha_pixels(test)
    

    display(test)
    display(np.array(test))

    display(sum)
    display(amount)
    display(sum / amount)
    # print(np.sum(np.array(image)))
    # print(np.sum(np.array(image.convert('L'))))
    # image_array = np.array(image.convert('L'))
    # display(image.convert('L'))
    # print(image_array)
    # np.sum(image_array)
    # print(np.sum(image_array))

# get_image_fitness(get_prepared_input_image())



# output_image = generate_new_image(OUTPUT_IMAGE_DIMENSIONS)
# n = generate_nail_positions(OUTPUT_IMAGE_RADIUS, OUTPUT_IMAGE_CENTER_POSITION, NAILS_AMOUNT)
#display(display_points(n, output_image))
# t = generate_thread_positions(n)


# g = get_threads_from_nail(t, n[0])



#display_lines(e, output_image)#.save('nails.png')

# Own algorithm

In [130]:


def get_thread_fitness(thread, ttp, input_image):
    # print(ttp[thread])
    # print[thread]

    fitness = 0
    # print(ttp[thread][0])
    for i in ttp[thread]:
        print(i, input_image.getpixel(i), input_image.size)
        fitness += input_image.getpixel(i)[0]
    return fitness / len(ttp[thread])
    
def get_best_fitting_thread(threads, ttp, input_image): # which thread cover the most black and the least white
    best_fitting_thread = None
    best_fitting_thread_fitness = 0 # might change to bigger values to avoid choosing threads that are too close to each other
    print(threads)
    for thread in threads:

        fitness = get_thread_fitness(thread, ttp, input_image)
        if fitness > best_fitting_thread_fitness:
            best_fitting_thread = thread
            best_fitting_thread_fitness = fitness
    return best_fitting_thread

In [131]:

# Start at random nail 
# Get all unused threds from that nail
# Put through fitness function
# Select best thread
# Add to used threads
# Go to next nail
# Repeat until fitness function returns positive values
# If no positive values, undo last nail and choose second best thread (parametrize how many step to go back)
# Repeat until no improvement can be made

# Save image

input_image = get_input_image()
output_image = generate_new_image(OUTPUT_IMAGE_DIMENSIONS)
nails = generate_nail_positions(OUTPUT_IMAGE_RADIUS, OUTPUT_IMAGE_CENTER_POSITION, NAILS_AMOUNT)
threads = generate_thread_positions(nails)
# display(nails)
# display(nails[0])

# display(threads)
# display(threads[0])

ntt = generate_nail_to_threads_dictionary(nails, threads)
ttp = generate_thread_to_points_dictionary(threads)

# display(ntt)
# display(ntt[nails[0]])

# display(ttp[threads[0]])

used_threads = []
current_nail = nails[0]
current_input_image = input_image.copy()
for i in range(0, 500):

    print("Current nail #", i, ": ", str(current_nail))

    # Make dictionary nail to threads
    current_threads = ntt[current_nail]

    print("Current threads: ", str(len(current_threads)))
    
    selected_thread = get_best_fitting_thread(current_threads, ttp, current_input_image)
    # selected_thread = current_threads[np.random.randint(0, len(current_threads))]

    used_threads.append(selected_thread)
    print("nnt len", len(ntt[current_nail]))
    ntt[current_nail].remove(selected_thread)
    ntt[get_other_end_of_thread(selected_thread, current_nail)].remove(selected_thread)
    print("nnt len", len(ntt[current_nail]))

    # ntt[current_nail] = [thread for thread in current_threads if thread != selected_thread]
    # ttp[selected_thread] = []
    # threads.remove(selected_thread)

    current_input_image = draw_line(current_input_image, selected_thread)

    # display(current_input_image)
    # display(display_threads(used_threads, output_image))

    current_nail = get_other_end_of_thread(selected_thread, current_nail)

current_input_image.save('nails.png')
display_threads(used_threads, output_image)


Current nail # 0 :  (500, 0)
Current threads:  97
[((500, 0), (562, 4)), ((500, 0), (593, 9)), ((500, 0), (624, 16)), ((500, 0), (654, 24)), ((500, 0), (683, 35)), ((500, 0), (712, 48)), ((500, 0), (740, 62)), ((500, 0), (767, 78)), ((500, 0), (793, 95)), ((500, 0), (818, 115)), ((500, 0), (841, 135)), ((500, 0), (864, 158)), ((500, 0), (884, 181)), ((500, 0), (904, 206)), ((500, 0), (921, 232)), ((500, 0), (937, 259)), ((500, 0), (951, 287)), ((500, 0), (964, 316)), ((500, 0), (975, 345)), ((500, 0), (983, 375)), ((500, 0), (990, 406)), ((500, 0), (995, 437)), ((500, 0), (998, 468)), ((500, 0), (999, 500)), ((500, 0), (998, 531)), ((500, 0), (995, 562)), ((500, 0), (990, 593)), ((500, 0), (983, 624)), ((500, 0), (975, 654)), ((500, 0), (964, 683)), ((500, 0), (951, 712)), ((500, 0), (937, 740)), ((500, 0), (921, 767)), ((500, 0), (904, 793)), ((500, 0), (884, 818)), ((500, 0), (864, 841)), ((500, 0), (841, 864)), ((500, 0), (818, 884)), ((500, 0), (793, 904)), ((500, 0), (767, 921)), 

IndexError: image index out of range